# Feature Extraction

In this notebook, useful features are extracted for the final classification from outputs from the denoised EEG signal datasets. In this the following two methods will be used.

[1] Time-Frequency Features - These are extracted using Wavelet Transformation where the parameters that will be analyzed would be following.

  Parameters to consider: 
  
    1. Mother Wavelet Function (Complex Morlet, DB, xx)
    2. Number of Levels (4, 6, 8)

[2] Alternative - Theoretical Comparison (No Implementartion)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q mne
!pip install -q wfdb pyEDFlib PyWavelets
!git clone https://github.com/raphaelvallat/entropy.git entropy/

     |████████████████████████████████| 7.6 MB 5.2 MB/s 
     |████████████████████████████████| 161 kB 4.8 MB/s 
     |████████████████████████████████| 2.4 MB 58.7 MB/s 
Cloning into 'entropy'...
remote: Enumerating objects: 1487, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 1487 (delta 0), reused 0 (delta 0), pack-reused 1477
Receiving objects: 100% (1487/1487), 3.38 MiB | 3.44 MiB/s, done.
Resolving deltas: 100% (953/953), done.


In [ ]:
import sys, os

sys.path.append(os.path.join(os.getcwd(), "drive/MyDrive/BME1473_Project"))

In [ ]:
import glob            # for file locations
import pandas as pd     # dataframes
import numpy as np
import pickle

import scipy.signal as signal
from pylab import *
from scipy.fft import fft, fftfreq, fftshift
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
RESULTS_DIR = './drive/MyDrive/BME1473_Project/Results'

Sub_Dir = ['F', 'S'] # Where F - Baseline Signals, S - Seizure Signal
sampling_freq = 173.73
dict_list = []

In [ ]:
def dict2dataframe(data_dict, passfreq = None, forder = None, window = None, data_type = 'nonfilt'):
  labels = []
  data_frame = pd.DataFrame()

  for data_dir in Sub_Dir:
    if data_type == 'nonfilt':
      sub_dict = data_dict[data_dir]
    elif data_type == 'firfilt':
      sub_dict = data_dict[data_dir][passfreq][forder][window]
    else:
      sub_dict = data_dict[data_dir][window]
      print(sub_dict.shape)
    labels += [data_dir for i in range(sub_dict.shape[0])]    

    for i in range(sub_dict.shape[0]):
      data_frame = data_frame.append(pd.Series(sub_dict[i, :]), ignore_index=True)

  data_frame['Label'] = labels

  return data_frame

## Wavelet Transformation

Wavelets can be used to analyse time series with non-stationary power at different frequency bands, express discontinuities caused by recording apparatus, and are useful for identifying and removing artefacts. 

Several oscillatory kernel-based wavelets are projected onto a signal, dividing the data into different frequency components which are each analysed in respect to their scale. A 'family' wavelet is a group of functions that is defined by stretching or shrinking a wavelet (dilation) and moving the wavelet to different positions in time (translation).

**Undecimated (Stationary) Wavelet Transform**

Unlike DWT, where an odd or even decimation can be made, UDWT uses both odd and even transformations at each scale. UDWT is a more computationally intensive method than DWT, but can result in better discrimination between noise and activity, as well as more precise frequency localization.

***DWT Features Extracted***

Based on the literature, 5 main features are derived from the wavelet transformation as follows:

    1.Log-Sum of Wavelet Transform
    2.Mean of Absolute Values of the coefficients in each sub-band
    3.Average Power of the wavelet coefficients in each sub-band
    4.Standard Deviation of the coefficients in each sub-band
    5.Ratio of the Absolute Mean values of adjacent sub-bands

In [ ]:
def data_index(feat_data, file_name, output = False):
    
    # get the file identifier from the file (e.g. F001)
    file_identifier = file_name.split('/')[-1].split('.')[0]
    # add this identifier to a column
    feat_data['file_id'] = file_identifier
    
    # if the file identifier has an S in...
    if re.findall('S', file_identifier):
        # make a class column with 'seizure' in
        feat_data['class'] = 'seizure'
    # ...otherwise...
    else:
        # .. make a class column with 'Baseline' in
        feat_data['class'] = 'baseline'
        
    
    # if the file identifier has a Z or O in...
    if re.findall('Z|O', file_identifier):
        # make a location column with 'surface' in
        feat_data['location'] = 'surface'
    # if the file identifier has an N in...
    elif re.findall('N', file_identifier):
        # make a location column with 'intracranial hippocampus' in
        feat_data['location'] = 'intracranial hippocampus'
    # if the file identifier has an S or F in...
    elif re.findall('F|S', file_identifier):
        # make a location column with 'intracranial epileptogenic zone' in
        feat_data['location'] = 'intracranial epileptogenic zone'
        
    # name the index
    feat_data.columns.name = 'feature'
    
    # add the file_id and class to the index
    feat_data = feat_data.set_index(['file_id', 'class', 'location'])
    # reorder the index so class is first, then file_id, then feature
    feat_data = feat_data.reorder_levels(['class', 'location', 'file_id'], axis='index')
    
    if output:
        display(feat_data)
        
    return feat_data

In [ ]:
sys.path.append(os.path.join(os.getcwd(), "./drive/MyDrive//BME1473_Project/Implementation"))
from FeatureExtraction import Seizure_Features

In [ ]:
def featExtraction( data_frame, 
                    sampling_freq, 
                    save_path,
                    downsample = 1,
                    window_size = None,
                    overlap = None,
                    weighted = False,
                    features = [],
                    bandpasses = [],
                    wavelet = 'db4',
                    wavelet_transform = 'DWT',
                    levels = 6,
                    fft_band = [1, 48],
                    scale = False
                   ):
  feature_df = pd.DataFrame()

  signal_data = data_frame.T.iloc[:-1].T
  nSignals, nSamples = signal_data.shape
  
  label_data = data_frame.T.iloc[-1].T

  for i_sig in range(nSignals):
      # get the signal from the dataframe
      signal_ = signal_data.iloc[i_sig].tolist()
      signal_ = pd.DataFrame(signal_)

      # setup the feature extraction function
      feat = Seizure_Features(  sf = sampling_freq, 
                                downsample = downsample,
                                window_size = window_size,
                                overlap = overlap,
                                weighted = weighted,
                                feature_list = features,   # ['power', 'power_ratio', 'mean', 'mean_abs', 'std', 'ratio', 'LSWT', 'fft_corr', 'fft_eigen', 'time_corr', 'time_eigen', 'sample_entropy', 'spectral_entropy']
                                bandpasses = bandpasses,
                                bandpass_mean = False,
                                bandpass_ratios=[[[3,12],[2,30]],],
                                wavelet = wavelet,
                                wavelet_transform = wavelet_transform,
                                levels = levels,
                                fft_band = fft_band,
                                scale = scale,
                            )

      # transform the data using the function
      part_x_feat = feat.transform(signal_.values, channel_names_list = ['CZ'])

      # put the numpy output back into a pandas df
      part_x_feat = pd.DataFrame(part_x_feat, columns = feat.feature_names)

      # re-index the data
      file_name = f'{label_data.iloc[i_sig]}_{(i_sig + 1) % 80 :02d}'

      part_x_feat = data_index(part_x_feat, file_name)

      # if there is no data in the feature data so far...
      if feature_df.empty:
          # then make this the feature dataframe...
          feature_df = part_x_feat
      else:
          # ...otherwise combine the two dataframes together down the index
          feature_df = pd.concat([feature_df, part_x_feat], axis='index')

  # reset the index into columns (for easy saving)
  feature_df_save = feature_df.reset_index()

  # save the dataframe to disk for later use
  os.makedirs(save_path, exist_ok=True)
  save_path = os.path.join(save_path, f'{wavelet}_{wavelet_transform}_{levels}.json.gzip')
  
  feature_df_save.to_json(save_path, 
                          orient='index', 
                          compression = 'gzip'
                         ) 
  return feature_df_save

In [ ]:
sub_dir = 'denoised'
dataset = 'train_zca'

file_path = os.path.join(RESULTS_DIR, sub_dir, dataset + '.pickle')

with open(file_path, 'rb') as file_:
  data_dict = pickle.load(file_)

In [ ]:
data_frame = dict2dataframe(data_dict)

### Feature extraction for different data sphering steps

In [ ]:
features = ['power', 'power_ratio', 'mean', 'mean_abs', 'std', 'ratio', 'LSWT', 'sample_entropy', 'spectral_entropy']
bandpasses = [[2,4],[4,8],[8,12], [12,30], [30, 48]]

wavelets = ['db4', 'db6', 'Haar']

wavelet_transforms = ['DWT', 'UDWT']
level = [6]

save_path = f'./drive/MyDrive/BME1473_Project/FeatureData/{sub_dir}/{dataset}'
os.makedirs(save_path, exist_ok = True)

for wavelet in wavelets:
  for wavelet_transform in wavelet_transforms:
    for levels in level:
      feature_dataframe = featExtraction(  data_frame = data_frame, 
                                          sampling_freq = sampling_freq, 
                                          save_path = save_path,
                                          downsample = 1,
                                          window_size = None,
                                          overlap = None,
                                          weighted = False,
                                          features = features,
                                          bandpasses = bandpasses,
                                          wavelet = wavelet,
                                          wavelet_transform = wavelet_transform,
                                          levels = levels,
                                          fft_band = [1, 48],
                                          scale = False
                                        )

In [ ]:
def filtfeatures(data_frame, sub_dir, dataset, passfreq, forder, window):
  wavelets = ['db4', 'db6', 'Haar']

  wavelet_transforms = ['DWT', 'UDWT']
  level = [6]
  save_path = f'./drive/MyDrive/BME1473_Project/FeatureData/{sub_dir}/{dataset}/{passfreq}_{forder}_{window}'
  os.makedirs(save_path, exist_ok = True)

  for wavelet in wavelets:
    for wavelet_transform in wavelet_transforms:
      for levels in level:
        feature_dataframe = featExtraction(  data_frame = data_frame, 
                                            sampling_freq = sampling_freq, 
                                            save_path = save_path,
                                            downsample = 1,
                                            window_size = None,
                                            overlap = None,
                                            weighted = False,
                                            features = features,
                                            bandpasses = bandpasses,
                                            wavelet = wavelet,
                                            wavelet_transform = wavelet_transform,
                                            levels = levels,
                                            fft_band = [1, 48],
                                            scale = False
                                          )
  return feature_dataframe

### Feature extraction for FIR filtered datasets

In [ ]:
sub_dir = 'fir_filtering'
dataset = 'fir_filt_original_test'

file_path = os.path.join(RESULTS_DIR, sub_dir, dataset + '.pickle')

with open(file_path, 'rb') as file_:
  data_dict = pickle.load(file_)

In [ ]:
pass_freqs = list(data_dict['S'].keys())
forders = list(data_dict['S'][pass_freqs[0]].keys())
windows = list(data_dict['S'][pass_freqs[0]][forders[0]].keys())

In [ ]:
print(pass_freqs)
print(forders)
print(windows)

[35, 40, 45, 50]
[16, 32, 64, 128]
['boxcar', 'hamming', 'blackman']


In [ ]:
for passfreq in pass_freqs[:-1]:
  for forder in forders:
    for window in windows:
      data_frame = dict2dataframe(data_dict, passfreq, forder, window, data_type = 'firfilt')
      feature_dataframe = filtfeatures(data_frame, sub_dir, dataset, passfreq, forder, window)


### Feature extraction for DWT denoised EEG signals

In [ ]:
def dwtfeatures(data_frame, sub_dir, dataset, dwt_):
  wavelets = ['db4', 'db6', 'Haar']

  wavelet_transforms = ['DWT', 'UDWT']
  level = [6]
  save_path = f'./drive/MyDrive/BME1473_Project/FeatureData/{sub_dir}/{dataset}/{dwt_}'
  os.makedirs(save_path, exist_ok = True)

  for wavelet in wavelets:
    for wavelet_transform in wavelet_transforms:
      for levels in level:
        feature_dataframe = featExtraction(  data_frame = data_frame, 
                                            sampling_freq = sampling_freq, 
                                            save_path = save_path,
                                            downsample = 1,
                                            window_size = None,
                                            overlap = None,
                                            weighted = False,
                                            features = features,
                                            bandpasses = bandpasses,
                                            wavelet = wavelet,
                                            wavelet_transform = wavelet_transform,
                                            levels = levels,
                                            fft_band = [1, 48],
                                            scale = False
                                          )
  return feature_dataframe

In [ ]:
sub_dir = 'dwt_filtering'
dataset = 'dwt_synth_original_train'

file_path = os.path.join(RESULTS_DIR, sub_dir, dataset + '.pickle')

with open(file_path, 'rb') as file_:
  data_dict = pickle.load(file_)

In [ ]:
wavelets = list(data_dict['S'].keys())

In [ ]:
for wavelet in wavelets:
  data_frame = dict2dataframe(data_dict, None, None, wavelet, data_type = 'dwtfilt')
  feature_dataframe = dwtfeatures(data_frame, sub_dir, dataset, wavelet)

(80, 4097)
(80, 4097)
(80, 4097)
(80, 4097)
(80, 4097)
(80, 4097)
(80, 4097)
(80, 4097)
(80, 4097)
(80, 4097)
